# Importing libraries

In [1]:
#Code for the obtention of
#some of the results of showed in Section 4 of
#https://doi.org/10.1016/j.cnsns.2020.105512
import numpy as np
import matplotlib.pyplot as plt
import time
import os

# Defining useful functions

In [3]:
def normal(x,sigma,mu):
    #normal distribution
    a = sigma*np.sqrt(np.pi*2)
    b = (x-mu)**2
    c = 2*sigma**2
    d = np.exp(-b/c)
    return d/a
    

def ut_t0(t,t0,ut0,pt0,qt0,rt0):
    #Numerical integration of the Riccati equation,
    #using the approximate solution presented in
    # https://doi.org/10.1115/1.4047990
    
    a1 = ut0 + (t-t0)*(pt0+ut0*qt0)
    a2 = 1-ut0*rt0*(t-t0)
    if a2 == 0:
        a2 += 1e-10
    
    return a1/a2


def moviemaker(tempos,x,UXT,path,X_1,W,option,v,FTT,num_images):
    
    ''' 
    Plots of the spatial distribution of the bacterial density
    along the rectangular channel at different time instants.
    More details of the experiment and the simulations in
    https://doi.org/10.1016/j.cnsns.2020.105512
    
    tempos: array with the time instants evaluated during the numerical
            integration of the set of Riccati equations
            
    x:      array with the spatial positions at which the equations were
            evaluated
            
    UXT:    array of arrays, spatial density distributions at different
            time instants
            
    path:   string, path where the file will be saved
    
    X_1:    positions of the oasis
    
    W:      width of the oasis
    
    option: choose a set of limits for axis
    
    v:      speed of the oasis
    
    FTT:    fraction of the time instants considered for plotting
    
    num_images: number of graphs to be plotted
    '''
    
    #determining limits for axis
    pos_tmax = int(FTT*len(tempos))+1
    
    if pos_tmax >= len(tempos):
        pos_tmax = len(tempos)
        xmax = x[len(x)-1]
    else:
        xmax = X_1[pos_tmax]
        
    
    
    if xmax == 0:
        print('Something went wrong')
        return 0
    
    diff = abs((x-xmax)/xmax)
    
    auxpos = np.where(diff == min(diff))
    pos_x_value = auxpos[0][0]
   
    z = np.array([max(UXT[i]) for i in range(pos_tmax)])
    k = np.array([min(UXT[i]) for i in range(pos_tmax)])
    umax = max(z)
    umin = min(k)

    counter = 0
    
    
    #time interval between images
    if pos_tmax > num_images:
        delta = int(pos_tmax/num_images)
    else:
        delta = 1
    
    #array with time instants to be plotted
    time_graph = tempos[0:pos_tmax+1:delta]

    #plotting
    for tv in range(len(time_graph)):
        
        for t in range(len(tempos)):
            
            if time_graph[tv] == tempos[t]:
 
                #the rectangle represents the oasis moving 
                #at speed v along the channel
                if option == 0:
                    plt.axis([x[0], x[pos_x_value], 
                              umin-0.01*umin, umax+0.01*umax])
                    rectangle = plt.Rectangle(
                        (X_1[t], 1e-15), W, umax+0.01*umax, fc='burlywood')
                
                else:
                    plt.axis([x[0], x[pos_x_value], 
                              min(UXT[t])-0.1*min(UXT[t]), 
                              max(UXT[t])+0.1*max(UXT[t])])
                    rectangle = plt.Rectangle((X_1[t], 1e-15),
                                              W, max(UXT[t])+0.1*max(UXT[t]), 
                                              fc='burlywood')
               
                plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
                plt.xlabel('x')
                plt.ylabel('U')        
                plt.plot(x,UXT[t])
        
                plt.legend(['t=%g' % tempos[t]])
        
                
                if v != 0:
                    plt.gca().add_patch(rectangle)
                
                plt.draw()
                plt.savefig(path+'/tmp_%04d.png' % counter)
                plt.close()
        
                counter += 1
                break

    return 0



def density_xt(tempos,Uxt,X_1):
    
    #filtering arrays for plotting
    #tempos, Uxt, X_1 are the same as above (moviemaker)


    if 1:
        
        #distance between selected points
        if len(tempos) > 10000:
            delta = int(len(tempos)/10000)
        else:
            delta = 1
        
        #array with time instants to be plotted
        time_graph = tempos[0:len(tempos):delta]
        time_graph[len(time_graph)-1] = tempos[len(tempos)-1]
        
        #densities and positions to be plotted
        dens = []
        xoasis = []
        
        
        for i in range(len(time_graph)):
            
            for j in range(len(tempos)):
    
                if time_graph[i] == tempos[j]: 
                    
                    dens.append(Uxt[j])
                    xoasis.append(X_1[j])
                    break
                
        time_graph = np.array(time_graph)
        dens = np.array(dens)
        xoasis = np.array(xoasis)


        return time_graph,dens,xoasis
        

        
def my_trapz(ds,new_Ut0):
    #numerical integration using the trapezoidal rule
    return ds*sum(new_Ut0)-ds*(new_Ut0[0]+new_Ut0[-1])/2

        
def I_omega(new_Ut0,x,x_value,alfabeta):
    
    '''
    Calculating the integrals of the working equation
    [Equations (10, 90, 91) of https://doi.org/10.1016/j.cnsns.2020.105512]
    
    new_Ut0: array of densities
    x: array of positions
    x_value: center of the interval to be integrated
    alfabeta: value of the correlation length
    '''
    xmin = x_value-alfabeta
    xmax = x_value+alfabeta
    
    area = 0

    auxpos = np.where(x == x_value)

    pos_x_value = auxpos[0][0]

    number_of_cells = int(alfabeta//abs(x[1]-x[0]))

    if number_of_cells == 0:
        return 0 
        
    pos_x_min = pos_x_value-number_of_cells
    pos_x_max = pos_x_value+number_of_cells
    
    dxx = abs(x[1]-x[0])
    
    #open boundary conditions. Density is null beyond the domain studied
    if xmin < 0:

        area = my_trapz(dxx,new_Ut0[0:pos_x_max+1])

    elif xmax > x[len(x)-1]:

        area = my_trapz(dxx,new_Ut0[pos_x_min:len(x)])

    else:
        
        area = my_trapz(dxx,new_Ut0[pos_x_min:pos_x_max+1])

        
    return area/(2*alfabeta)

    
def URiccati(nUt0,x,x_value,W,x1,epsilon,alfa,beta,dt,t0,v):
    '''
    Numerical integration of the Riccati equation, 
    [Equation (91) of https://doi.org/10.1016/j.cnsns.2020.105512]
    Parameters
    ----------
    nUt0 : array of densities
    x : positions
    x_value : center of the interval to be integrated
    W : width of the oasis
    x1 : position of the oasis
    epsilon : characterize the intensity of radiation
              outside the oasis
    alfa : growth length parameter
    beta : competition length parameter
    dt : integration step
    t0 : initial instant
    v : speed of the oasis

    Returns
    -------
    newU : densities at the instant t0+dt

    '''
    
    auxpos_x_value = np.where(x==x_value)
    
        
    pos_x_value = auxpos_x_value[0][0]
    
        
    Utx = nUt0[pos_x_value]
    
    dx = abs(x[1]-x[0])
    
    #open boundary conditions
    if pos_x_value == 0:
        ump1 = nUt0[pos_x_value+1]
        umm1 = 0
    elif pos_x_value == len(nUt0)-1:
        ump1 = 0
        umm1 = nUt0[pos_x_value-1]
    else:
        ump1 = nUt0[pos_x_value+1]
        umm1 = nUt0[pos_x_value-1]
        
    Ibeta = I_omega(nUt0,x,x_value,beta)  
    
    D = alfa**2/6 #Diffusion coefficient
    
    if x_value >= x1 and x_value < x1+W:
        growth = 1
    else:
        growth = -epsilon

    
    p = D*(ump1+umm1)/dx**2
    
    q = growth-2*D/dx**2-Ibeta+dx*Utx/(2*beta)
    
    r = -dx/(2*beta)
    
    newU = ut_t0(t0+dt,t0,Utx,p,q,r) 
   
    return newU 


    
def new_U(mUt0,x,W,x1,epsilon,alfa,beta,t0,v,tmax,dt):
    #calculating the density distribution at the instant t0+dt
    #starting from the density at t0 (mUt0)
    U = np.zeros(len(mUt0))

    for i in range(len(mUt0)):
        
        U[i] = URiccati(mUt0,x,x[i],W,x1,epsilon,alfa,beta,dt,t0,v)
        #density is a non negative quantity
        if U[i] < 0:
            U[i] = 0
            
    return U


    
def Simulation_exp(zUt0,xU,W,x10,epsilon,alfa,beta,
                   t0,v,contador,total,Idist,tmax,dt):
    '''
    Following the movement of the oasis along the channel
    subjected to ultraviolet radiation

    Parameters
    ----------
    zUt0 : densities at t=t0
    xU : position
    W : width of the oasis
    x10 : initial position of the left side of the oasis
    epsilon : characterize radiation intensity outside the oasis
    alfa : alpha
    beta : beta
    t0 : initial instant
    v : speed of the oasis
    contador : counter
    total : total of runs
    Idist : strin
    tmax : max time
    dt : integration step

    Returns
    -------
    None.

    '''
    t0global = time.time()

    dx = xU[1]-xU[0]
    
    #strings for folder's names
    str_dx = ("%0.3g"%(dx))
    str_W = ("%g"%(W))
    str_tmax = ("%0.2g"%(tmax))
   
    str_v = ("%0.3g"%(v))
    str_ep = ("%g"%(epsilon))
    str_alfa =("%g"%(alfa)) 
    str_beta =("%g"%(beta)) 
    
    #creating folders to save simulation results according to
    #parameter values
    
    paths = os.getcwd()
    path_image = paths+'/Simulation_of_experiment'
    folderName = path_image.split('/')
    
    try:  
        os.mkdir(path_image)
    except OSError:  
        print ("Creation of the directory %s failed" % folderName[-1])
    else:  
        print ("Successfully created the directory %s " % folderName[-1])


    path_image += '/alfa_'+str_alfa+'beta_'+str_beta+'W_'+str_W+'epsilon_'+str_ep
    folderName = path_image.split('/')
    
    try:  
        os.mkdir(path_image)
    except OSError:  
        print ("Creation of the directory %s failed" % folderName[-1])
    else:  
        print ("Successfully created the directory %s " % folderName[-1])
      

    path_image += '/v_'+str_v+'_tmax_'+str_tmax+'_dx_'+str_dx
    folderName = path_image.split('/')
    
    try:  
        os.mkdir(path_image)
    except OSError:  
        print ("Creation of the directory %s failed" % folderName[-1])
    else:  
        print ("Successfully created the directory %s " % folderName[-1])
             
    Uxt = [] #densities
    Times = [] #time instants
    X_1 = [] #oasis positions
    
    
    
    Uxt.append(zUt0)
    
    Times.append(t0)
    X_1.append(x10)
    
    Uaux = zUt0

    
    if 1:
        
        t = t0
        x1 = x10
        cont = 1
        iterations = 0
        
        
        if 1:
            
            while (1):
                
                #numerical integration from t0 to tmax
                newU = new_U(Uaux,xU,W,x1,epsilon,alfa,beta,t,v,tmax,dt)
                
                x1 = x10+v*(t+dt)
                t += dt
                iterations += 1


                alarm = 0
                for rev in range(len(Uaux)):
                    if newU[rev] != newU[rev]:
                        alarm = 1
                        break
                    
                if alarm == 1:
                    break
    

                Uxt.append(newU)
                Times.append(t)
                X_1.append(x1)
                cont += 1
                
                if t > tmax:
                    break
                    
                Uaux = newU

    Times = np.array(Times)
    X_1 = np.array(X_1)
    
    time_graph,density,xoasis = density_xt(Times,Uxt,X_1)
    
    #saving some data     
    save_data = open(path_image+"/general_data.dat", "w")
    save_data.write("Calculo Terminado \n")
    save_data.write("Tiempo de Calculo: %g segundos \n"%(time.time()-t0global))    
    
        
    np.savez_compressed(path_image+'/density'+'.npz',density)
    np.savez_compressed(path_image+'/time_for_density'+'.npz',time_graph)
    np.savez_compressed(path_image+'/position_oasis'+'.npz',xoasis)
        
        
    path_image2 = path_image+'/images'
    folderName = path_image2.split('/')
        
    try:  
        os.mkdir(path_image2)
    except OSError:  
        print ("Creation of the directory %s failed" % folderName[-1])
    else:  
        print ("Successfully created the directory %s " % folderName[-1])  
    
    #plotting        
    moviemaker(time_graph,xU,density,path_image2,xoasis,W,0,v,1,100)  
       
      
    path_image2 = path_image+'/images2'
    folderName = path_image2.split('/')
        
    try:  
        os.mkdir(path_image2)
    except OSError:  
        print ("Creation of the directory %s failed" % folderName[-1])
    else:  
        print ("Successfully created the directory %s " % folderName[-1])  
    
    #plotting        
    moviemaker(time_graph,xU,density,path_image2,xoasis,W,1,v,1,100) 

    np.savez_compressed(path_image+'/xU'+'.npz',xU)
    np.savez_compressed(path_image+'/Ut0'+'.npz',zUt0)
   
    print('Data saved')    
    
    string2Print = 'iterations: %0.2g  tmax: %0.2g '+\
        'dt: %0.2g t: %0.2g  x1: %0.2g  \n'
    save_data.write(string2Print%(iterations,tmax,dt,t,x1))
        
    save_data.write("Data Saved \n")
    save_data.close()    



# Making the calculations

In [4]:

#Experimental values
a1 = 6e-4 #growth rate
b1 = 7.93e-11  #competigion rate
D = 1.6e-4  #diffusion coefficient
vf = np.sqrt(2*D*a1) #speed of the oasis
L1 = 25  #length of the channel
W1 = 3  #width of the oasis
alfa_exp = np.sqrt(6*D/a1) #growth length parameter


#Dimensionless values, according to the transformations
#presented in https://doi.org/10.1016/j.cnsns.2020.105512
a = 1
b = 1
vf_ = vf/(L1*a1)
L = 1
sigma = 0.01*L
W = 0.3 #width of the oasis
pontos = 100 #number of equations to be integrated

#alpha values
alfa = np.array([0.05])

#beta values
beta = np.array([0.20])


t0 = 0 #initial instant
xxU = np.linspace(0,L,pontos)  #space points
dx1 = xxU[1]-xxU[0]

#initial position of the left side of the oasis
x10 = 0

#radiation intensities
epsilon = np.array([0])

#oasis speeds
v = np.array([0.25*vf_])


total=len(epsilon)*len(v)*len(alfa)*len(beta)


        
print('Calculation started')
_t0 = time.time()
contador = 1  

#intensity of radiation outside the oasis  
for j in range(len(epsilon)):

    #velocities
    for k in range(len(v)):
        
        
        if v[k] == 0:
            print('Speed should be higher than 0')
            break
        else:
            tmax = (L-W-x10)/(v[k])
            myep = epsilon[j]
        
        #alpha values
        for m in range(len(alfa)):
        
            #beta values
            for p in range(len(beta)):
                
                #initial distribution under the oasis
                Ut0 = (normal(xxU,sigma,x10+W/2))/25
                initial_dist = 'normal_sigma'+str(sigma)
                
                dt = 0.01

                Simulation_exp(Ut0,xxU,W,x10,myep,alfa[m],beta[p],
                               t0,v[k],contador,total,initial_dist,tmax,dt)
                contador += 1
_tf = time.time()                
print(f'Calculation finished after {round((_tf-_t0)/60, 2)} minutes.')  

Calculation started
Successfully created the directory Simulation_of_experiment 
Successfully created the directory alfa_0.05beta_0.2W_0.3epsilon_0 
Successfully created the directory v_0.0073_tmax_96_dx_0.0101 
Successfully created the directory images 
Successfully created the directory images2 
Data saved
Calculation finished after 2.21 minutes.
